<a href="https://colab.research.google.com/github/AllswellSam/AllswellSam/blob/main/research%20codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad, acorr_ljungbox
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

# load data
df = pd.read_csv("RD.csv")

# Parse date if present
for c in ["date","Date","DATES","DAY","day","DATE","dt","timestamp","Timestamp"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")
        df = df.sort_values(by=c)
        df = df.rename(columns={c: "date"})
        break

# Standardize key column names used below (edit if your headers differ)
df = df.rename(columns={
    'Inflation':'inflation',
    'Policy Rate':'policy_rate',
    'Daily new confirmed cases of COVID-19 per million':'cases_per_million',
    'Stringency index (weighted average)':'stringency',
    'USD/GHS Exchange Rate':'exr_usd_ghs'
})

key = ['exr_usd_ghs','inflation','policy_rate','cases_per_million','stringency']
df = df.dropna(subset=key)
for k in key:
    df[k] = pd.to_numeric(df[k], errors='coerce')
df = df.dropna(subset=key).copy()

# 2) Quick summaries
print("\n== Summary stats ==")
print(df[key].describe().T)

print("\n== Correlation matrix ==")
print(df[key].corr())

# Stationarity on level exchange rate
adf_exr = adfuller(df['exr_usd_ghs'], autolag='AIC')
print("\nADF on level EXR: stat=%.4f, p=%.6f" % (adf_exr[0], adf_exr[1]))

# 3) (Appendix-only) OLS on LEVELS with diagnostics
Y = df['exr_usd_ghs']
X = sm.add_constant(df[['stringency','cases_per_million','inflation','policy_rate']])
ols_levels = sm.OLS(Y, X).fit()
print("\n== OLS on levels (appendix only; not for inference) ==")
print(ols_levels.summary())

dw = durbin_watson(ols_levels.resid)
bp = het_breuschpagan(ols_levels.resid, ols_levels.model.exog) # lm stat, lm p, f stat, f p
norm_stat, norm_p = normal_ad(ols_levels.resid)

print(f"\nDiagnostics (levels): DW={dw:.3f}, BP p(LM)={bp[1]:.4g}, BP p(F)={bp[3]:.4g}, Normality p={norm_p:.4g}")

# VIF (exclude constant)
vif = []
for i, name in enumerate(X.columns[1:], start=1):
    vif.append([name, variance_inflation_factor(X.values, i)])
vif_df = pd.DataFrame(vif, columns=["variable","VIF"])
print("\nVIF (levels):\n", vif_df)

# 4) Correct model: returns (Δln EXR) with HAC (Newey–West)
df2 = df.copy()
df2['ln_exr'] = np.log(df2['exr_usd_ghs'])
df2['dln_exr'] = df2['ln_exr'].diff()
for c in ['inflation','policy_rate','stringency','cases_per_million']:
    df2['d_'+c] = df2[c].diff()

mod = df2.dropna(subset=['dln_exr','d_inflation','d_policy_rate','d_stringency','d_cases_per_million']).copy()
Yd = mod['dln_exr']
Xd = sm.add_constant(mod[['d_stringency','d_cases_per_million','d_inflation','d_policy_rate']])

ols_diff_hac = sm.OLS(Yd, Xd).fit(cov_type='HAC', cov_kwds={'maxlags':7})
print("\n== Δ ln(EXR) with HAC (preferred) ==")
print(ols_diff_hac.summary())
adf_d = adfuller(mod['dln_exr'], autolag='AIC')
print("ADF on Δ ln(EXR): stat=%.4f, p=%.6g" % (adf_d[0], adf_d[1]))

# 5) Robustness: add AR(1) lag of Δ ln(EXR)
mod['dln_exr_lag1'] = mod['dln_exr'].shift(1)
mod2 = mod.dropna(subset=['dln_exr_lag1']).copy()
Yda = mod2['dln_exr']
Xda = sm.add_constant(mod2[['dln_exr_lag1','d_stringency','d_cases_per_million','d_inflation','d_policy_rate']])

ols_diff_ar1_hac = sm.OLS(Yda, Xda).fit(cov_type='HAC', cov_kwds={'maxlags':7})
print("\n== Δ ln(EXR) + AR(1) with HAC (robustness) ==")
print(ols_diff_ar1_hac.summary())

# 6) Plots (no custom colors/styles)
# Time-series (if date exists)
if 'date' in df.columns:
    plt.figure(figsize=(10,4))
    plt.plot(df['date'], df['exr_usd_ghs'])
    plt.title("USD/GHS Exchange Rate Over Time")
    plt.xlabel("Date"); plt.ylabel("USD/GHS")
    plt.tight_layout(); plt.savefig("plot_exr_timeseries.png"); plt.close()

    plt.figure(figsize=(10,4))
    plt.plot(df['date'], df['stringency'])
    plt.title("COVID-19 Stringency Index Over Time")
    plt.xlabel("Date"); plt.ylabel("Stringency Index")
    plt.tight_layout(); plt.savefig("plot_stringency_timeseries.png"); plt.close()

    plt.figure(figsize=(10,4))
    plt.plot(df['date'], df['cases_per_million'])
    plt.title("Daily New COVID-19 Cases per Million")
    plt.xlabel("Date"); plt.ylabel("Cases per Million")
    plt.tight_layout(); plt.savefig("plot_cases_timeseries.png"); plt.close()

# Residual plots for levels (appendix)
plt.figure(figsize=(6,5))
plt.scatter(ols_levels.fittedvalues, ols_levels.resid, s=10)
plt.axhline(0, linestyle="--")
plt.title("Residuals vs Fitted (Levels OLS)")
plt.xlabel("Fitted values"); plt.ylabel("Residuals")
plt.tight_layout(); plt.savefig("plot_resid_vs_fitted_levels.png"); plt.close()

fig = qqplot(ols_levels.resid, line='s')
plt.title("Q–Q Plot of Residuals (Levels OLS)")
plt.tight_layout(); plt.savefig("plot_resid_qq_levels.png"); plt.close()

# Residual plots for preferred diff model
plt.figure(figsize=(6,5))
plt.scatter(ols_diff_hac.fittedvalues, ols_diff_hac.resid, s=10)
plt.axhline(0, linestyle="--")
plt.title("Residuals vs Fitted (Diff + HAC)")
plt.xlabel("Fitted values"); plt.ylabel("Residuals")
plt.tight_layout(); plt.savefig("plot_resid_vs_fitted_diff.png"); plt.close()

fig = qqplot(ols_diff_hac.resid, line='s')
plt.title("Q–Q Plot of Residuals (Diff + HAC)")
plt.tight_layout(); plt.savefig("plot_resid_qq_diff.png"); plt.close()

print("\nDone. Files written to current folder.")


== Summary stats ==
                    count       mean        std    min       25%       50%  \
exr_usd_ghs        1066.0   6.781604   1.968877   5.32   5.78000   5.87105   
inflation          1066.0  16.731520  11.684368   7.50   9.94129  10.87871   
policy_rate        1066.0  16.177767   3.497956  13.50  14.50000  14.50000   
cases_per_million  1066.0   4.839630   8.556907   0.00   0.00000   0.81450   
stringency         1066.0  42.839015  14.214413   0.00  38.89000  42.91000   

                         75%       max  
exr_usd_ghs         7.093750  14.75000  
inflation          20.529032  54.10000  
policy_rate        17.000000  27.00000  
cases_per_million   5.693962  81.51038  
stringency         50.640000  84.26000  

== Correlation matrix ==
                   exr_usd_ghs  inflation  policy_rate  cases_per_million  \
exr_usd_ghs           1.000000   0.956808     0.955420          -0.198323   
inflation             0.956808   1.000000     0.974801          -0.205793   
policy_

In [6]:
from statsmodels.stats.diagnostic import het_breuschpagan, normal_ad
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import acorr_ljungbox
# --- DIAGNOSTICS for DIFFERENCED HAC + AR(1) MODEL ---
print("\n== Diagnostics: Δ ln(EXR) + AR(1) with HAC ==")

dw_ar = durbin_watson(ols_diff_ar1_hac.resid)
bp_lm_ar, bp_lm_p_ar, bp_f_ar, bp_f_p_ar = het_breuschpagan(ols_diff_ar1_hac.resid, ols_diff_ar1_hac.model.exog)
ad_stat_ar, ad_p_ar = normal_ad(ols_diff_ar1_hac.resid)
lb_ar_df = acorr_ljungbox(ols_diff_ar1_hac.resid, lags=[5,10], return_df=True)

print(f"Durbin–Watson: {dw_ar:.6f}")
print(f"Anderson–Darling normality: stat={ad_stat_ar:.6f}, p={ad_p_ar:.6g}")
print(f"Breusch–Pagan: LM p={bp_lm_p_ar:.3g}, F p={bp_f_p_ar:.3g}")
print("Ljung–Box p-values (lags 5, 10):", list(lb_ar_df['lb_pvalue'].values))


== Diagnostics: Δ ln(EXR) + AR(1) with HAC ==
Durbin–Watson: 1.963903
Anderson–Darling normality: stat=inf, p=0
Breusch–Pagan: LM p=1.32e-17, F p=2.41e-18
Ljung–Box p-values (lags 5, 10): [np.float64(0.0002621991113314027), np.float64(0.00037088708407989927)]


In [3]:
# ---- Five-panel dashboard: EXR, Inflation, Policy, Stringency, Cases ----
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateLocator, AutoDateFormatter

# Safety check: ensure 'date' exists
if 'date' not in df.columns:
    raise ValueError("No 'date' column found. Parse/rename your date column to 'date' first.")

fig, axes = plt.subplots(5, 1, figsize=(12, 12), sharex=True)

series_info = [
    ("USD/GHS Exchange Rate", "exr_usd_ghs", "USD/GHS"),
    ("Inflation Rate", "inflation", "Inflation (%)"),
    ("Monetary Policy Rate", "policy_rate", "Policy Rate (%)"),
    ("COVID-19 Stringency Index", "stringency", "Index (0–100)"),
    ("Daily New COVID-19 Cases per Million", "cases_per_million", "Cases / Million"),
]

for ax, (title, col, ylabel) in zip(axes, series_info):
    ax.plot(df['date'], df[col])
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.grid(True, which="both", linestyle="--", alpha=0.3)

# X-axis date formatting
locator = AutoDateLocator()
formatter = AutoDateFormatter(locator)
axes[-1].xaxis.set_major_locator(locator)
axes[-1].xaxis.set_major_formatter(formatter)
axes[-1].set_xlabel("Date")

plt.tight_layout()
plt.savefig("figure_dashboard_exr_inflation_policy_stringency_cases.png", dpi=300)
plt.close()
print("Saved 5-panel dashboard to figure_dashboard_exr_inflation_policy_stringency_cases.png")


Saved 5-panel dashboard to figure_dashboard_exr_inflation_policy_stringency_cases.png
